In [6]:
import re
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import joblib  # For model saving and loading
import pickle
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
import time
import pickle
import re
import joblib
from collections import Counter
import math

class ClassificationModelPipeline:
    def __init__(self, data_path, target='T0', ftrz = ['Y','M','D','W','H','S','SNODP','SWGDN','LWGAB','T2M','SWLAND','GHTSKIN','HFLUX','SPEED','TLML','TSH','EVPSOIL','LWLAND','TS','QV2M','SLP'], seed=42, test_size=0.2, n_splits=2, export=1, use_smote=False):
        self.data_path = data_path
        self.prdHrz = int(re.search(r'(\d+)Days', self.data_path).group(1))
        self.loc = str(re.search(r'_(\w+)_', self.data_path).group(1))
        self.target = target
        self.ftrz = ftrz
        self.seed = seed
        self.test_size = test_size
        self.n_splits = n_splits
        self.export = export
        self.use_smote = use_smote
        self.models = {
            "Random Forest Classifier": RandomForestClassifier(n_jobs=-1, random_state=seed),
#             "Extra Trees Classifier": ExtraTreesClassifier(n_jobs=-1, random_state=seed),
            "SGD Classifier": SGDClassifier(random_state=seed),
            "KNeighbors Classifier": KNeighborsClassifier(),
            "Support Vector Classifier": SVC(),
            "MLP Classifier": MLPClassifier(random_state=seed, max_iter=1000),
#             "HistGradientBoosting Classifier": HistGradientBoostingClassifier(random_state=seed)
        }
        self.results = pd.DataFrame(columns=["Model", "Train Time (s)", "Train wAccuracy", "Train f1w", "Train FAR", "Test Time (s)", "Test wAccuracy", "Test f1w", "Test FAR"])

    def load_and_prepare_data(self):
        print('1. Loading data', end=' ')
        data = pd.read_csv(self.data_path, header=0, usecols= [self.target] + self.ftrz )
        if ~all(isinstance(x, int) for x in data[self.target]):
            data[f"{self.target}"] = pd.cut(data[f'{self.target}'],bins=[-math.inf,0,+math.inf],labels=[0,1])

        print(f"(#samples={len(data)}, #features={data.loc[:, data.columns != self.target].shape[1]}, target='{self.target}')")
    
        # Splitting data into features and target
        X = data.loc[:, data.columns != self.target].values
        y = data[self.target].values

        # Splitting into train and test sets
        print('2. Splitting data', end=' ')
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.test_size, random_state=self.seed, stratify=y)
        print(f'(#training samples={len(X_train)}|{Counter(y_train)}, #testing samples={len(X_test)}|{Counter(y_test)})')

        # Export data and scaler if specified
        if self.export == 1:
            print('2-1. Exporting data...')
            train_data = pd.DataFrame(X_train, columns=data.columns[data.columns != self.target])
            train_data[self.target] = y_train
            test_data = pd.DataFrame(X_test, columns=data.columns[data.columns != self.target])
            test_data[self.target] = y_test

            if self.use_smote == 0:
                train_data.to_csv(f'FTstates_{self.loc}_{self.target}_Trainset_{self.prdHrz}Days.csv', index=False)
                test_data.to_csv(f'FTstates_{self.loc}_{self.target}_Testset_{self.prdHrz}Days.csv', index=False)
            else:
                train_data.to_csv(f'FTstates_{self.loc}_{self.target}_Trainset_{self.prdHrz}Days_SMOTE.csv', index=False)
                test_data.to_csv(f'FTstates_{self.loc}_{self.target}_Testset_{self.prdHrz}Days_SMOTE.csv', index=False)               

        # Standardizing features
        print('3. Standardizing X...')
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        self.X_train, self.X_test, self.y_train, self.y_test = X_train, X_test, y_train, y_test
        self.scaler = scaler
        self.data = data

        # Handling imbalance with SMOTE if specified
        if self.use_smote:
            print('4. Applying SMOTE for class balancing...',end=' ')
            smote = SMOTE(random_state=self.seed)
            self.X_train, self.y_train = smote.fit_resample(X_train, y_train)
            print(f'before {Counter(y_train)}| after {Counter(self.y_train)}')

        # Export data and scaler if specified
        if self.export == 1:
            print('5. Exporting scaler...')
            if self.use_smote == 0:
                with open(f'FTstates_{self.loc}_{self.target}_StdScaler_{self.prdHrz}Days.pkl', 'wb') as f:
                    pickle.dump(self.scaler, f)
            else:
                with open(f'FTstates_{self.loc}_{self.target}_StdScaler_{self.prdHrz}Days_SMOTE.pkl', 'wb') as f:
                    pickle.dump(self.scaler, f)                    

    def A_initial_benchmark(self):
        print(f'6. Initial benchmark | Model learning:')
        for name, model in self.models.items():
            print(f'\t-{name}', end=' ')
            if self.n_splits == 1:
                start_train = time.time()
                model.fit(self.X_train, self.y_train)
                end_train = time.time()
                y_train_pred = model.predict(self.X_train)
            else:
                kf = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=self.seed)
                y_train_pred = cross_val_predict(model, self.X_train, self.y_train, cv=kf)
                start_train = time.time()
                model.fit(self.X_train, self.y_train)
                end_train = time.time()

            start_test = time.time()
            y_test_pred = model.predict(self.X_test)
            end_test = time.time()

            train_accuracy = balanced_accuracy_score(self.y_train, y_train_pred)
            test_accuracy = balanced_accuracy_score(self.y_test, y_test_pred)
            
            # Compute False Alarm Rates
            tn, fp, fn, tp = confusion_matrix(self.y_train, y_train_pred).ravel()
            train_FAR = fp / (fp + tn)
            train_f1 = f1_score(self.y_train, y_train_pred, average='weighted')

            tn, fp, fn, tp = confusion_matrix(self.y_test, y_test_pred).ravel()
            test_FAR = fp / (fp + tn)
            test_f1 = f1_score(self.y_test, y_test_pred, average='weighted')
            
            train_time = end_train - start_train
            test_time = end_test - start_test

            self.results = pd.concat([self.results, pd.DataFrame({
                "Model": [name],
                "Train Time (s)": [train_time],
                "Train wAccuracy": [train_accuracy],
                "Train f1w": [train_f1],
                "Train FAR": [train_FAR],
                "Test Time (s)": [test_time],
                "Test wAccuracy": [test_accuracy],
                "Test f1w": [test_f1],
                "Test FAR": [test_FAR]
            })], ignore_index=True)
            print(f'[Test Accuracy={test_accuracy}, FAR={test_FAR}]')

    def X_SelectFeatures_ML(self):
        self.results_B = self.results
        best_model_name = 'Extra Trees Classifier'#self.results.iloc[0]['Model']
        best_model = ExtraTreesClassifier(n_jobs=-1, random_state=self.seed)#self.models[best_model_name]
#         self.X_train = self.X_train[:, [self.data.columns.get_loc(c) for c in ['GHTSKIN','H','M','S','SNODP','W','Y'] if c in self.data]]
#         self.X_test = self.X_test[:, [self.data.columns.get_loc(c) for c in ['GHTSKIN','H','M','S','SNODP','W','Y'] if c in self.data]]
        print(f'5. {best_model_name} baseline | Model learning:',end=' ')
        if self.n_splits == 1:
            start_train = time.time()
            best_model.fit(self.X_train, self.y_train)
            end_train = time.time()
            y_train_pred = best_model.predict(self.X_train)
        else:
            kf = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=self.seed)
            y_train_pred = cross_val_predict(best_model, self.X_train, self.y_train, cv=kf)
            start_train = time.time()
            best_model.fit(self.X_train, self.y_train)
            end_train = time.time()

        start_test = time.time()
        y_test_pred = best_model.predict(self.X_test)
        end_test = time.time()
        
        # Export predictions
        print(f'7. Exporting {best_model_name} and test data...')
        with open(f'FTstates_{self.loc}_{self.target}_ET_{self.prdHrz}Days.pkl', 'wb') as f:
            pickle.dump(best_model, f)
        X_test_unstandardized = self.scaler.inverse_transform(self.X_test)
        tmp1 = pd.DataFrame(X_test_unstandardized, columns=self.ftrz)
        tmp = pd.DataFrame({
            'Targets': self.y_test,
            'Predicted': y_test_pred
        })
        tmp = pd.concat([tmp1, tmp], axis=1)
        tmp.to_csv(f'FTstates_{self.loc}_{self.target}_testresults_{self.prdHrz}Days.csv', index=False)


        train_accuracy = balanced_accuracy_score(self.y_train, y_train_pred)
        test_accuracy = balanced_accuracy_score(self.y_test, y_test_pred)
        
        # Compute False Alarm Rates
        tn, fp, fn, tp = confusion_matrix(self.y_train, y_train_pred).ravel()
        train_FAR = fp / (fp + tn)
        train_f1 = f1_score(self.y_train, y_train_pred, average='weighted')

        tn, fp, fn, tp = confusion_matrix(self.y_test, y_test_pred).ravel()
        test_FAR = fp / (fp + tn)
        test_f1 = f1_score(self.y_test, y_test_pred, average='weighted')
        
        train_time = end_train - start_train
        test_time = end_test - start_test

        self.results = pd.concat([self.results, pd.DataFrame({
            "Model": [best_model_name],
            "Train Time (s)": [train_time],
            "Train wAccuracy": [train_accuracy],
            "Train f1w": [train_f1],
            "Train FAR": [train_FAR],
            "Test Time (s)": [test_time],
            "Test wAccuracy": [test_accuracy],
            "Test f1w": [test_f1],
            "Test FAR": [test_FAR]
        })], ignore_index=True)
        print(f'[Test Accuracy={test_accuracy}, FAR={test_FAR}]')
        
        
    def display_results(self):
        if not self.results.empty:
            self.results = self.results.sort_values(by="Test wAccuracy", ascending=False)
            display(self.results)

    def display_results_unsorted(self):
        if not self.results.empty:
#             self.results = self.results.sort_values(by="Test wAccuracy", ascending=False)
            display(self.results)
            
    def save_best_model(self):
        best_model_name = self.results.iloc[0]['Model']
        best_model = self.models[best_model_name]
        path = f'classification_{self.prdHrz}Days_bestmodel.pkl'
        joblib.dump(best_model, path)
        print(f"Best model '{best_model_name}' saved to '{path}'")

    def load_model(self):
        with open(f'classification_StdScaler_{self.prdHrz}Days.pkl', 'rb') as f:
            loaded_scaler = pickle.load(f)
        return joblib.load(f'classification_{self.prdHrz}Days_bestmodel.pkl')


In [3]:

if __name__ == "__main__":
    for hrz in list([0,7,30,90]):
        data_path=f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_{hrz}Days.csv' 
        print(f'T0|Horizon={hrz}:')
        pipeline = ClassificationModelPipeline(data_path=data_path, target='T0',  ftrz=['Y', 'M', 'D', 'W', 'SWGDN', 'T2M', 'GHTSKIN', 'HFLUX', 'SPEED'], seed=42, test_size=0.2, n_splits=10, export=0, use_smote=False)
        pipeline.load_and_prepare_data()
        pipeline.X_SelectFeatures_ML()
        pipeline.display_results()

#     for hrz in list([0,7,30,90]):
#         print(f'T8|Horizon={hrz}:')
#         data_path=f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T8_{hrz}Days.csv' 
#         pipeline = ClassificationModelPipeline(data_path=data_path, target='T8', ftrz=['Y', 'S', 'SNODP', 'SWLAND', 'GHTSKIN', 'SPEED', 'TLML', 'EVPSOIL', 'SLP'], seed=42, test_size=0.2, n_splits=10, export=0, use_smote=False)
#         pipeline.load_and_prepare_data()
#         pipeline.X_SelectFeatures_ML()
#         pipeline.display_results()

#     for hrz in list([0,7,30,90]):
#         print(f'T16|Horizon={hrz}:')
#         data_path=f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_{hrz}Days.csv' 
#         pipeline = ClassificationModelPipeline(data_path=data_path, target='T16', ftrz=['Y', 'M', 'D', 'S', 'SNODP', 'GHTSKIN', 'HFLUX', 'TLML', 'SLP'], seed=42, test_size=0.2, n_splits=10, export=0, use_smote=False)
#         pipeline.load_and_prepare_data()
#         pipeline.X_SelectFeatures_ML()
#         pipeline.display_results()


T8|Horizon=0:
1. Loading data (#samples=6080, #features=9, target='T8')
2. Splitting data (#training samples=4864|Counter({0: 3232, 1: 1632}), #testing samples=1216|Counter({0: 808, 1: 408}))
3. Standardizing X...
5. Extra Trees Classifier baseline | Model learning: 7. Exporting Extra Trees Classifier and test data...
[Test Accuracy=0.9815691127936323, FAR=0.009900990099009901]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,Extra Trees Classifier,0.149009,0.978648,0.981687,0.012067,0.029918,0.981569,0.984361,0.009901


T8|Horizon=7:
1. Loading data (#samples=6071, #features=9, target='T8')
2. Splitting data (#training samples=4856|Counter({0: 3226, 1: 1630}), #testing samples=1215|Counter({0: 807, 1: 408}))
3. Standardizing X...
5. Extra Trees Classifier baseline | Model learning: 7. Exporting Extra Trees Classifier and test data...
[Test Accuracy=0.942775530286464, FAR=0.04337050805452292]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,Extra Trees Classifier,0.155088,0.946112,0.950845,0.039678,0.030286,0.942776,0.947419,0.043371


T8|Horizon=30:
1. Loading data (#samples=6048, #features=9, target='T8')
2. Splitting data (#training samples=4838|Counter({0: 3206, 1: 1632}), #testing samples=1210|Counter({0: 802, 1: 408}))
3. Standardizing X...
5. Extra Trees Classifier baseline | Model learning: 7. Exporting Extra Trees Classifier and test data...
[Test Accuracy=0.9180235685296563, FAR=0.05610972568578554]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,Extra Trees Classifier,0.156592,0.917533,0.925645,0.057704,0.030243,0.918024,0.926468,0.05611


T8|Horizon=90:
1. Loading data (#samples=5988, #features=9, target='T8')
2. Splitting data (#training samples=4790|Counter({0: 3158, 1: 1632}), #testing samples=1198|Counter({0: 790, 1: 408}))
3. Standardizing X...
5. Extra Trees Classifier baseline | Model learning: 7. Exporting Extra Trees Classifier and test data...
[Test Accuracy=0.9527860511293125, FAR=0.04050632911392405]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,Extra Trees Classifier,0.168638,0.951272,0.954574,0.038632,0.030963,0.952786,0.955054,0.040506


In [14]:

if __name__ == "__main__":
    for hrz in list([0,7,30,90]):
        data_path=f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_{hrz}Days.csv' 
        print(f'T0|Horizon={hrz}:')
        pipeline = ClassificationModelPipeline(data_path=data_path, target='T0',  ftrz=['Y','M','H','S','SNODP','GHTSKIN','TSH'], seed=42, test_size=0.2, n_splits=10, export=0, use_smote=False)
        pipeline.load_and_prepare_data()
        pipeline.X_SelectFeatures_ML()
        pipeline.display_results()

    for hrz in list([0,7,30,90]):
        print(f'T8|Horizon={hrz}:')
        data_path=f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_07_{hrz}Days.csv' 
        pipeline = ClassificationModelPipeline(data_path=data_path, target='T_07', ftrz=['Y','M','D','SNODP','EVPSOIL'], seed=42, test_size=0.2, n_splits=10, export=0, use_smote=False)
        pipeline.load_and_prepare_data()
        pipeline.X_SelectFeatures_ML()
        pipeline.display_results()

    for hrz in list([0,7,30,90]):
        print(f'T16|Horizon={hrz}:')
        data_path=f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_{hrz}Days.csv' 
        pipeline = ClassificationModelPipeline(data_path=data_path, target='T_12', ftrz=['Y','M','D','H'], seed=42, test_size=0.2, n_splits=10, export=0, use_smote=False)
        pipeline.load_and_prepare_data()
        pipeline.X_SelectFeatures_ML()
        pipeline.display_results()


T0|Horizon=0:
1. Loading data (#samples=4987, #features=7, target='T0')
2. Splitting data (#training samples=3989|Counter({0: 2820, 1: 1169}), #testing samples=998|Counter({0: 706, 1: 292}))
3. Standardizing X...
5. Extra Trees Classifier baseline | Model learning: 7. Exporting Extra Trees Classifier and test data...
[Test Accuracy=0.9685329271605417, FAR=0.018413597733711047]


/tmp/ipykernel_1892454/1659378207.py:206: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,Extra Trees Classifier,0.147125,0.977155,0.980461,0.014894,0.03499,0.968533,0.973948,0.018414


T0|Horizon=7:
1. Loading data (#samples=4980, #features=7, target='T0')
2. Splitting data (#training samples=3984|Counter({0: 2815, 1: 1169}), #testing samples=996|Counter({0: 704, 1: 292}))
3. Standardizing X...
5. Extra Trees Classifier baseline | Model learning: 7. Exporting Extra Trees Classifier and test data...
[Test Accuracy=0.9696353518057286, FAR=0.01278409090909091]


/tmp/ipykernel_1892454/1659378207.py:206: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


...

/tmp/ipykernel_1892454/1659378207.py:206: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,Extra Trees Classifier,0.143762,0.980824,0.984633,0.011486,0.035514,0.986744,0.991914,0.002703


T16|Horizon=90:
1. Loading data (#samples=4897, #features=4, target='T_12')
2. Splitting data (#training samples=3917|Counter({0: 2912, 1: 1005}), #testing samples=980|Counter({0: 728, 1: 252}))
3. Standardizing X...
5. Extra Trees Classifier baseline | Model learning: 7. Exporting Extra Trees Classifier and test data...
[Test Accuracy=0.9747405372405373, FAR=0.006868131868131868]


/tmp/ipykernel_1892454/1659378207.py:206: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,Extra Trees Classifier,0.137276,0.985775,0.989024,0.007555,0.035298,0.974741,0.983609,0.006868


## Benchmark with other conventional ML models

In [7]:

if __name__ == "__main__":
    for hrz in list([0,7,30,90]):
        data_path=f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_{hrz}Days.csv' 
        print(f'T0|Horizon={hrz}:')
        pipeline = ClassificationModelPipeline(data_path=data_path, target='T0',  ftrz=['Y', 'M', 'D', 'W', 'SWGDN', 'T2M', 'GHTSKIN', 'HFLUX', 'SPEED'], seed=42, test_size=0.2, n_splits=10, export=0, use_smote=False)
        pipeline.load_and_prepare_data()
        pipeline.A_initial_benchmark()
        pipeline.display_results_unsorted()

    for hrz in list([0,7,30,90]):
        print(f'T8|Horizon={hrz}:')
        data_path=f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T8_{hrz}Days.csv' 
        pipeline = ClassificationModelPipeline(data_path=data_path, target='T8', ftrz=['Y', 'S', 'SNODP', 'SWLAND', 'GHTSKIN', 'SPEED', 'TLML', 'EVPSOIL', 'SLP'], seed=42, test_size=0.2, n_splits=10, export=0, use_smote=False)
        pipeline.load_and_prepare_data()
        pipeline.A_initial_benchmark()
        pipeline.display_results_unsorted()

    for hrz in list([0,7,30,90]):
        print(f'T16|Horizon={hrz}:')
        data_path=f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_{hrz}Days.csv' 
        pipeline = ClassificationModelPipeline(data_path=data_path, target='T16', ftrz=['Y', 'M', 'D', 'S', 'SNODP', 'GHTSKIN', 'HFLUX', 'TLML', 'SLP'], seed=42, test_size=0.2, n_splits=10, export=0, use_smote=False)
        pipeline.load_and_prepare_data()
        pipeline.A_initial_benchmark()
        pipeline.display_results_unsorted()


T0|Horizon=0:
1. Loading data (#samples=6080, #features=9, target='T0')
2. Splitting data (#training samples=4864|Counter({0: 3190, 1: 1674}), #testing samples=1216|Counter({0: 798, 1: 418}))
3. Standardizing X...
6. Initial benchmark | Model learning:
	-Random Forest Classifier 

/tmp/ipykernel_518840/2259918216.py:141: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


[Test Accuracy=0.9785828206880839, FAR=0.021303258145363407]
	-SGD Classifier [Test Accuracy=0.9757917521075415, FAR=0.012531328320802004]
	-KNeighbors Classifier [Test Accuracy=0.9570517202096149, FAR=0.021303258145363407]
	-Support Vector Classifier [Test Accuracy=0.9733994076099339, FAR=0.012531328320802004]
	-MLP Classifier [Test Accuracy=0.9714057871952608, FAR=0.021303258145363407]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,Random Forest Classifier,0.180812,0.975466,0.978600,0.014420,0.032499,0.978583,0.978666,0.021303
1,SGD Classifier,0.005520,0.972062,0.974714,0.019436,0.000491,0.975792,0.979411,0.012531
2,KNeighbors Classifier,0.003927,0.956188,0.963083,0.021317,0.044324,0.957052,0.963709,0.021303
3,Support Vector Classifier,0.080273,0.970374,0.974871,0.015047,0.023136,0.973399,0.977751,0.012531
4,MLP Classifier,4.622818,0.975593,0.978402,0.015361,0.001375,0.971406,0.973699,0.021303


T0|Horizon=7:
1. Loading data (#samples=6071, #features=9, target='T0')
2. Splitting data (#training samples=4856|Counter({0: 3184, 1: 1672}), #testing samples=1215|Counter({0: 797, 1: 418}))
3. Standardizing X...
6. Initial benchmark | Model learning:
	-Random Forest Classifier 

...

,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,Random Forest Classifier,0.187917,0.961073,0.964691,0.028386,0.033492,0.959404,0.967630,0.016030
1,SGD Classifier,0.011317,0.912540,0.918732,0.070349,0.000786,0.919685,0.925894,0.062885
2,KNeighbors Classifier,0.003881,0.932265,0.938712,0.049059,0.044326,0.928893,0.939492,0.039457
3,Support Vector Classifier,0.138391,0.943445,0.950174,0.036717,0.041138,0.946356,0.954442,0.029593
4,MLP Classifier,5.676270,0.955129,0.960511,0.029003,0.001416,0.947569,0.956855,0.024661


T16|Horizon=90:
1. Loading data (#samples=5988, #features=9, target='T16')
2. Splitting data (#training samples=4790|Counter({0: 3193, 1: 1597}), #testing samples=1198|Counter({0: 799, 1: 399}))
3. Standardizing X...
6. Initial benchmark | Model learning:
	-Random Forest Classifier 

/tmp/ipykernel_518840/2259918216.py:141: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


[Test Accuracy=0.9705615728934351, FAR=0.016270337922403004]
	-SGD Classifier [Test Accuracy=0.8696961427348096, FAR=0.03504380475594493]
	-KNeighbors Classifier [Test Accuracy=0.9217019394543932, FAR=0.03629536921151439]
	-Support Vector Classifier [Test Accuracy=0.9492598831245824, FAR=0.01877346683354193]
	-MLP Classifier [Test Accuracy=0.9605396469898149, FAR=0.02127659574468085]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,Random Forest Classifier,0.265031,0.960860,0.965339,0.025681,0.065218,0.970562,0.974926,0.016270
1,SGD Classifier,0.010823,0.864112,0.889910,0.053241,0.000470,0.869696,0.899193,0.035044
2,KNeighbors Classifier,0.003855,0.926574,0.936790,0.042280,0.044296,0.921702,0.935320,0.036295
3,Support Vector Classifier,0.158548,0.932055,0.943170,0.033824,0.044548,0.949260,0.959694,0.018773
4,MLP Classifier,6.250063,0.947240,0.953822,0.032884,0.001386,0.960540,0.966547,0.021277


In [8]:

if __name__ == "__main__":
    for hrz in list([0,7,30,90]):
        data_path=f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_{hrz}Days.csv' 
        print(f'T0|Horizon={hrz}:')
        pipeline = ClassificationModelPipeline(data_path=data_path, target='T0',  ftrz=['Y','M','H','S','SNODP','GHTSKIN','TSH'], seed=42, test_size=0.2, n_splits=10, export=0, use_smote=False)
        pipeline.load_and_prepare_data()
        pipeline.A_initial_benchmark()
        pipeline.display_results_unsorted()

    for hrz in list([0,7,30,90]):
        print(f'T07|Horizon={hrz}:')
        data_path=f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_07_{hrz}Days.csv' 
        pipeline = ClassificationModelPipeline(data_path=data_path, target='T_07', ftrz=['Y','M','D','SNODP','EVPSOIL'], seed=42, test_size=0.2, n_splits=10, export=0, use_smote=False)
        pipeline.load_and_prepare_data()
        pipeline.A_initial_benchmark()
        pipeline.display_results_unsorted()

    for hrz in list([0,7,30,90]):
        print(f'T12|Horizon={hrz}:')
        data_path=f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_{hrz}Days.csv' 
        pipeline = ClassificationModelPipeline(data_path=data_path, target='T_12', ftrz=['Y','M','D','H'], seed=42, test_size=0.2, n_splits=10, export=0, use_smote=False)
        pipeline.load_and_prepare_data()
        pipeline.A_initial_benchmark()
        pipeline.display_results_unsorted()

T0|Horizon=0:
1. Loading data (#samples=4987, #features=7, target='T0')
2. Splitting data (#training samples=3989|Counter({0: 2820, 1: 1169}), #testing samples=998|Counter({0: 706, 1: 292}))
3. Standardizing X...
6. Initial benchmark | Model learning:
	-Random Forest Classifier 

/tmp/ipykernel_518840/2259918216.py:141: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


[Test Accuracy=0.9626877255617214, FAR=0.019830028328611898]
	-SGD Classifier [Test Accuracy=0.952889130350421, FAR=0.032577903682719546]
	-KNeighbors Classifier [Test Accuracy=0.9608589778415926, FAR=0.026912181303116147]
	-Support Vector Classifier [Test Accuracy=0.9594425472466918, FAR=0.029745042492917848]
	-MLP Classifier [Test Accuracy=0.9602671814971477, FAR=0.021246458923512748]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,Random Forest Classifier,0.258320,0.973483,0.978430,0.014539,0.032806,0.962688,0.969909,0.019830
1,SGD Classifier,0.004352,0.950743,0.959378,0.028369,0.000134,0.952889,0.959019,0.032578
2,KNeighbors Classifier,0.003138,0.960789,0.966211,0.026241,0.028337,0.960859,0.965999,0.026912
3,Support Vector Classifier,0.051003,0.969290,0.973948,0.019504,0.015386,0.959443,0.964034,0.029745
4,MLP Classifier,3.326302,0.967183,0.973407,0.017730,0.000921,0.960267,0.967903,0.021246


T0|Horizon=7:
1. Loading data (#samples=4980, #features=7, target='T0')
2. Splitting data (#training samples=3984|Counter({0: 2815, 1: 1169}), #testing samples=996|Counter({0: 704, 1: 292}))
3. Standardizing X...
6. Initial benchmark | Model learning:
	-Random Forest Classifier 

...

,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,Random Forest Classifier,0.177149,0.979153,0.983127,0.012838,0.032959,0.983451,0.990885,0.001351
1,SGD Classifier,0.009037,0.500639,0.639564,0.002703,0.000130,0.500000,0.637432,0.000000
2,KNeighbors Classifier,0.002043,0.955537,0.967417,0.020270,0.021452,0.970785,0.979785,0.010811
3,Support Vector Classifier,0.116053,0.940963,0.944087,0.054392,0.041859,0.927338,0.937159,0.054054
4,MLP Classifier,4.934266,0.951501,0.962464,0.026351,0.000900,0.952209,0.967569,0.016216


T16|Horizon=90:
1. Loading data (#samples=4897, #features=4, target='T_12')
2. Splitting data (#training samples=3917|Counter({0: 2912, 1: 1005}), #testing samples=980|Counter({0: 728, 1: 252}))
3. Standardizing X...
6. Initial benchmark | Model learning:
	-Random Forest Classifier 

/tmp/ipykernel_518840/2259918216.py:141: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


[Test Accuracy=0.9766483516483517, FAR=0.01098901098901099]
	-SGD Classifier [Test Accuracy=0.4952686202686203, FAR=0.04120879120879121]
	-KNeighbors Classifier [Test Accuracy=0.9528388278388278, FAR=0.01098901098901099]
	-Support Vector Classifier [Test Accuracy=0.9342948717948718, FAR=0.04807692307692308]
	-MLP Classifier [Test Accuracy=0.9313186813186813, FAR=0.03021978021978022]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,Random Forest Classifier,0.180170,0.983424,0.986986,0.009272,0.032910,0.976648,0.982642,0.010989
1,SGD Classifier,0.009876,0.497834,0.633770,0.010302,0.000129,0.495269,0.635162,0.041209
2,KNeighbors Classifier,0.002021,0.965976,0.976932,0.011332,0.021484,0.952839,0.970147,0.010989
3,Support Vector Classifier,0.116465,0.947740,0.949634,0.049794,0.035187,0.934295,0.943352,0.048077
4,MLP Classifier,4.754792,0.956329,0.966561,0.022665,0.000985,0.931319,0.949835,0.030220


## Different training approaches

In [102]:
import re
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import joblib  # For model saving and loading
import pickle
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
import time
import pickle
import re
import joblib
from collections import Counter
import math

class FlexibleDatasetClassificationPipeline:
    def __init__(self, data_path1, data_path2=None, ftrz1=None, ftrz2=None, combine_datasets=False, use_second_for_testing=False, 
                 seed=42, test_size=0.2, n_splits=2, export=1, use_smote=False, depths=['T0', 'T_07', 'T_12'], test_depth=None):
        # Default features if not provided
        if ftrz1 is None:
            ftrz1 = ['Y', 'M', 'D', 'W', 'H', 'S', 'SNODP', 'SWGDN', 'LWGAB', 'T2M', 'SWLAND', 
                     'GHTSKIN', 'HFLUX', 'SPEED', 'TLML', 'TSH', 'EVPSOIL', 'LWLAND', 'TS', 'QV2M', 'SLP']           
            
        self.data_path1 = data_path1
        self.data_path2 = data_path2
        self.combine_datasets = combine_datasets
        self.use_second_for_testing = use_second_for_testing
        self.seed = seed
        self.test_size = test_size
        self.n_splits = n_splits
        self.export = export
        self.use_smote = use_smote
        self.depths = depths # all considered depths 
        self.test_depth = test_depth  # Depth to be used as test data in agnostic depth learning

        # Use ftrz2 only when combine_datasets is True
        if self.combine_datasets & self.use_second_for_testing==False :
            if ftrz2 is None:
                ftrz2 = ['Y', 'M', 'D', 'W', 'H', 'S', 'SNODP', 'SWGDN', 'LWGAB', 'T2M', 'SWLAND', 
                     'GHTSKIN', 'HFLUX', 'SPEED', 'TLML', 'TSH', 'EVPSOIL', 'LWLAND', 'TS', 'QV2M', 'SLP']
            self.ftrz = list(set(ftrz1).union(set(ftrz2)))  # Union of features from both datasets
        elif self.combine_datasets & self.use_second_for_testing:
            if ftrz2 is None:
                ftrz2 = ['Y', 'M', 'D', 'W', 'H', 'S', 'SNODP', 'SWGDN', 'LWGAB', 'T2M', 'SWLAND', 
                     'GHTSKIN', 'HFLUX', 'SPEED', 'TLML', 'TSH', 'EVPSOIL', 'LWLAND', 'TS', 'QV2M', 'SLP']
            self.ftrz = ftrz2  # only the second dataset's features are used!            
        else:
            self.ftrz = ftrz1  # Use only ftrz1 for separate dataset processing

        self.models = {
            "ET": ExtraTreesClassifier(n_jobs=-1, random_state=seed),
            "SGD Classifier": SGDClassifier(random_state=seed),
            "KNeighbors Classifier": KNeighborsClassifier(),
            "Support Vector Classifier": SVC(),
            "MLP Classifier": MLPClassifier(random_state=seed, max_iter=1000)
        }
        self.results = pd.DataFrame(columns=["Model", "Train Time (s)", "Train wAccuracy", "Train f1w", 
                                             "Train FAR", "Test Time (s)", "Test wAccuracy", "Test f1w", "Test FAR"])

    def extract_target_from_path(self, file_path):
        # Use re.search to find the first match in the file path
        match = re.search(r'T(_?\d+)', file_path)

        # If a match is found, return the matched group (i.e., the target)
        if match:
            return match.group(0)
        else:
            raise ValueError(f"no hrz was detected from the provided paths.")

    def load_data(self, data_path):
        print(f'Loading data from {data_path}, ', end='')
        # Extract the target from both file paths
        target = self.extract_target_from_path(data_path)
        self.target = target
        print(f'target is {self.target}, ', end='')        
        data = pd.read_csv(data_path, header=0, usecols=[self.target] + self.ftrz)
        if ~all(isinstance(x, int) for x in data[self.target]):
            data[self.target] = pd.cut(data[self.target], bins=[-math.inf, 0, +math.inf], labels=[0, 1])

        print(f"(#samples={len(data)}, #features={data.loc[:, data.columns != self.target].shape[1]}, target='{self.target}')")
        
        X = data.loc[:, data.columns != self.target].values
        y = data[self.target].values
        return X, y, data

    def process_data(self, X, y, scaler=None, standardize=True):
        if standardize:
            if scaler is None:
#                 print('Standardizing X, ', end='')
                scaler = StandardScaler()
                X = scaler.fit_transform(X)
            else:
                X = scaler.transform(X)
            return X, y, scaler
        return X, y, None

    def split_and_process_data(self, X, y, prevscaler=None):
        # Split into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.test_size,
                                                            random_state=self.seed, stratify=y)
        # Process training data
        X_train, y_train, scaler = self.process_data(X_train, y_train, standardize=True)
        if prevscaler == None:
            # Standardize testing data using the same scaler
            X_test, y_test, _ = self.process_data(X_test, y_test, scaler=scaler, standardize=True)
        else:
            # Standardize testing data using previous scaler
            X_test, y_test, _ = self.process_data(X_test, y_test, scaler=prevscaler, standardize=True)
            
        return X_train, y_train, X_test, y_test, scaler

    def process_combined_data(self):
        print("Combining datasets...")
        # Load and concatenate both datasets
        X1, y1, _ = self.load_data(self.data_path1)
        X2, y2, _ = self.load_data(self.data_path2)
        
        # Combine the datasets
        X_combined = np.vstack((X1, X2))
        y_combined = np.hstack((y1, y2))
        
        return self.split_and_process_data(X_combined, y_combined)

    def process_separate_data(self):
        print("Loading separate datasets, ",end='')
        # Load training data
        X1, y1, _ = self.load_data(self.data_path1)
#         print(f'X1 size is {X1.shape} | y1 size is {y1.shape}')
        # Load testing data
        X2, y2, _ = self.load_data(self.data_path2)
#         print(f'X2 size is {X2.shape} | y2 size is {y2.shape}')

        print("Standardizing separate datasets")
        # Standardize training data
        X_train, y_train, _, _, scaler = self.split_and_process_data(X1, y1)
        _, _, X_test, y_test, _ = self.split_and_process_data(X2, y2, scaler)
#         print(f"X_train {X_train.shape}, y_train {y_train.shape}, X_test {X_test.shape}, y_test {y_test.shape}")

        return X_train, y_train, X_test, y_test, scaler

    def process_agnostic_depth_data(self):
        print("Performing agnostic learning for depths (same location learning but all depths, test on a specific depth though) ...")

        # Define the depths to use for training and testing
        depths = ['T0', 'T_07', 'T_12']
        
        # Initialize lists for training data across all depths
        all_X_train, all_y_train, all_depth_train = [], [], []

        for depth in self.depths:
            # Load and split data for each depth| self.data_path1 should always be 'T0'
            depth_data_path = self.data_path1.replace('T0', depth)
            X, y, _ = self.load_data(depth_data_path)
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=self.test_size, random_state=self.seed, stratify=y
            )

            # Identify depth in the training data
            depth_train_column = np.full(X_train.shape[0], int(re.findall(r'\d+', depth)[0]))
            all_X_train.append(X_train)
            all_y_train.append(y_train)
            all_depth_train.append(depth_train_column)

            # Use test data only if it matches the specified test depth
            if depth == self.test_depth:
                test_depth_column = np.full(X_test.shape[0], int(re.findall(r'\d+', depth)[0]))
                X_test_final, y_test_final, depth_test_final = X_test, y_test, test_depth_column

        # Concatenate train data from all depths
        X_train_combined = np.vstack(all_X_train)
        y_train_combined = np.hstack(all_y_train)
        depth_train_combined = np.hstack(all_depth_train).reshape(-1, 1)

        # Include depth as a feature in the training and test sets
        X_train_with_depth = np.hstack([X_train_combined, depth_train_combined])
        depth_test_final = depth_test_final.reshape(-1, 1)
        X_test_with_depth = np.hstack([X_test_final, depth_test_final])
        
        # Standardize the training and test data using the same scaler
        X_train_with_depth, y_train_combined, scaler = self.process_data(X_train_with_depth, y_train_combined, standardize=True)
        print(X_train_with_depth.shape, '|', X_train_combined.shape, '|', depth_train_combined.shape)
        X_test_with_depth, y_test_final, _ = self.process_data(X_test_with_depth, y_test_final, scaler=scaler, standardize=True)
        print(X_test_with_depth.shape, '|', X_test_final.shape, '|', depth_test_final.shape)

        return X_train_with_depth, y_train_combined, X_test_with_depth, y_test_final, scaler    
    
    def train_and_test_model(self, model_name="ET"):
        if model_name not in self.models:
            raise ValueError(f"Model '{model_name}' not found in available models.")

        if self.combine_datasets & pd.isna(self.test_depth):
            X_train, y_train, X_test, y_test, scaler = self.process_combined_data()
        elif self.combine_datasets==False:
            if pd.isna(self.test_depth):
                X_train, y_train, X_test, y_test, scaler = self.process_separate_data()
            else:
                X_train, y_train, X_test, y_test, scaler = self.process_agnostic_depth_data()
        
        model = self.models[model_name]

        # Train the model
        print(f'Training {model} model...', end=' ')
        kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=self.seed)
        y_train_pred = cross_val_predict(model, X_train, y_train, cv=kf)
        start_train = time.time()
        model.fit(X_train, y_train)
        end_train = time.time()

        # Test the model
        start_test = time.time()
        y_test_pred = model.predict(X_test)
        end_test = time.time()

        train_accuracy = balanced_accuracy_score(y_train, model.predict(X_train))
        test_accuracy = balanced_accuracy_score(y_test, y_test_pred)

        # Compute False Alarm Rates
        tn, fp, fn, tp = confusion_matrix(y_train, model.predict(X_train)).ravel()
        train_FAR = fp / (fp + tn)
        train_f1 = f1_score(y_train, model.predict(X_train), average='weighted')

        tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
        test_FAR = fp / (fp + tn)
        test_f1 = f1_score(y_test, y_test_pred, average='weighted')

        train_time = end_train - start_train
        test_time = end_test - start_test

        # Save results
        self.results = pd.concat([self.results, pd.DataFrame({
            "Model": [model_name],
            "Train Time (s)": [train_time],
            "Train wAccuracy": [train_accuracy],
            "Train f1w": [train_f1],
            "Train FAR": [train_FAR],
            "Test Time (s)": [test_time],
            "Test wAccuracy": [test_accuracy],
            "Test f1w": [test_f1],
            "Test FAR": [test_FAR]
        })], ignore_index=True)
        print(f'[Test Accuracy={test_accuracy}, FAR={test_FAR}]')

    def display_results(self):
        if not self.results.empty:
            self.results = self.results.sort_values(by="Test wAccuracy", ascending=False)
            display(self.results)


In [103]:
#Toolik
for hrz in list([0,7,30,90]):
    print(f'Ground|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_{hrz}Days.csv'  # Dataset 1 (Deadhorse)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        ftrz1=['Y', 'M', 'D', 'W', 'SWGDN', 'T2M', 'GHTSKIN', 'HFLUX', 'SPEED'], #toolik features
        seed=42,
        test_size=0.2,
        depths=['T0', 'T8', 'T16'], 
        test_depth='T0'
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()

print('*****************************************************************************************************************')
for hrz in list([0,7,30,90]):
    print(f'Ground|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_{hrz}Days.csv'  # Dataset 1 (Deadhorse)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        ftrz1=['Y', 'S', 'SNODP', 'SWLAND', 'GHTSKIN', 'SPEED', 'TLML', 'EVPSOIL', 'SLP'], #toolik features
        seed=42,
        test_size=0.2,
        depths=['T0', 'T8', 'T16'], 
        test_depth='T8'
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
        
print('*****************************************************************************************************************')
for hrz in list([0,7,30,90]):
    print(f'Ground|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_{hrz}Days.csv'  # Dataset 1 (Deadhorse)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        ftrz1=['Y', 'M', 'D', 'S', 'SNODP', 'GHTSKIN', 'HFLUX', 'TLML', 'SLP'], #toolik features
        seed=42,
        test_size=0.2,
        depths=['T0', 'T8', 'T16'], 
        test_depth='T16'
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
                

Ground|Horizon=0:
Performing agnostic learning for depths (same location learning but all depths, test on a specific depth though) ...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_0Days.csv, target is T0, (#samples=6080, #features=21, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T8_0Days.csv, target is T8, (#samples=6080, #features=21, target='T8')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_0Days.csv, target is T16, (#samples=6080, #features=21, target='T16')
(14592, 22) | (14592, 21) | (14592, 1)
(1216, 22) | (1216, 21) | (1216, 1)
Training ExtraTreesClassifier(n_jobs=-1, random_state=42) model... [Test Accuracy=0.9879813169286853, FAR=0.002506265664160401]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.176925,1.0,1.0,0.0,0.029836,0.987981,0.990936,0.002506


Ground|Horizon=7:
Performing agnostic learning for depths (same location learning but all depths, test on a specific depth though) ...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_7Days.csv, target is T0, (#samples=6071, #features=21, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T8_7Days.csv, target is T8, (#samples=6071, #features=21, target='T8')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_7Days.csv, target is T16, (#samples=6071, #features=21, target='T16')
(14568, 22) | (14568, 21) | (14568, 1)
(1215, 22) | (1215, 21) | (1215, 1)
Training ExtraTreesClassifier(n_jobs=-1, random_state=42) model... [Test Accuracy=0.9838224081934047, FAR=0.0012547051442910915]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.181589,1.0,1.0,0.0,0.030141,0.983822,0.988437,0.001255


Ground|Horizon=30:
Performing agnostic learning for depths (same location learning but all depths, test on a specific depth though) ...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_30Days.csv, target is T0, (#samples=6048, #features=21, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T8_30Days.csv, target is T8, (#samples=6048, #features=21, target='T8')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_30Days.csv, target is T16, (#samples=6048, #features=21, target='T16')
(14514, 22) | (14514, 21) | (14514, 1)
(1210, 22) | (1210, 21) | (1210, 1)
Training ExtraTreesClassifier(n_jobs=-1, random_state=42) model... [Test Accuracy=0.9837838571760467, FAR=0.0037926675094816687]


...

,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.170334,1.0,1.0,0.0,0.031995,0.983958,0.981971,0.022059


Ground|Horizon=30:
Performing agnostic learning for depths (same location learning but all depths, test on a specific depth though) ...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_30Days.csv, target is T0, (#samples=6048, #features=21, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T8_30Days.csv, target is T8, (#samples=6048, #features=21, target='T8')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_30Days.csv, target is T16, (#samples=6048, #features=21, target='T16')
(14514, 22) | (14514, 21) | (14514, 1)
(1210, 22) | (1210, 21) | (1210, 1)
Training ExtraTreesClassifier(n_jobs=-1, random_state=42) model... [Test Accuracy=0.9844864936694386, FAR=0.018495684340320593]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.180829,1.0,1.0,0.0,0.029298,0.984486,0.983522,0.018496


Ground|Horizon=90:
Performing agnostic learning for depths (same location learning but all depths, test on a specific depth though) ...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_90Days.csv, target is T0, (#samples=5988, #features=21, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T8_90Days.csv, target is T8, (#samples=5988, #features=21, target='T8')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_90Days.csv, target is T16, (#samples=5988, #features=21, target='T16')
(14370, 22) | (14370, 21) | (14370, 1)
(1198, 22) | (1198, 21) | (1198, 1)
Training ExtraTreesClassifier(n_jobs=-1, random_state=42) model... [Test Accuracy=0.9818397683821569, FAR=0.016270337922403004]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.193003,1.0,1.0,0.0,0.031499,0.98184,0.982498,0.01627


In [99]:
#Deadhorse
for hrz in list([0,7,30,90]):
    print(f'Ground|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_{hrz}Days.csv'  # Dataset 1 (Deadhorse)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        ftrz1=['Y', 'M', 'H', 'S', 'SNODP', 'GHTSKIN', 'TSH'], #deadhorse features
        seed=42,
        test_size=0.2,
        depths=['T0', 'T_07', 'T_12'], 
        test_depth='T0'
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()

print('*****************************************************************************************************************')
for hrz in list([0,7,30,90]):
    print(f'Ground|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_{hrz}Days.csv'  # Dataset 1 (Deadhorse)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        ftrz1=['Y','M','D','SNODP','EVPSOIL'], #deadhorse features
        seed=42,
        test_size=0.2,
        depths=['T0', 'T_07', 'T_12'], 
        test_depth='T_07'
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
        
print('*****************************************************************************************************************')
for hrz in list([0,7,30,90]):
    print(f'Ground|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_{hrz}Days.csv'  # Dataset 1 (Deadhorse)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        ftrz1=['Y', 'M', 'D', 'H'], #deadhorse features
        seed=42,
        test_size=0.2,
        depths=['T0', 'T_07', 'T_12'], 
        test_depth='T_12'
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
                

Ground|Horizon=0:
Performing agnostic learning for depths (same location learning but all depths, test on a specific depth though) ...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_0Days.csv, target is T0, (#samples=4987, #features=21, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_07_0Days.csv, target is T_07, (#samples=4987, #features=21, target='T_07')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_0Days.csv, target is T_12, (#samples=4987, #features=21, target='T_12')
(11967, 22) | (11967, 21) | (11967, 1)
(998, 22) | (998, 21) | (998, 1)
Training ExtraTreesClassifier(n_jobs=-1, random_state=42) model... [Test Accuracy=0.9729616981644611, FAR=0.019830028328611898]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.184298,1.0,1.0,0.0,0.031379,0.972962,0.976,0.01983


Ground|Horizon=7:
Performing agnostic learning for depths (same location learning but all depths, test on a specific depth though) ...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_7Days.csv, target is T0, (#samples=4980, #features=21, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_07_7Days.csv, target is T_07, (#samples=4980, #features=21, target='T_07')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_7Days.csv, target is T_12, (#samples=4980, #features=21, target='T_12')
(11952, 22) | (11952, 21) | (11952, 1)
(996, 22) | (996, 21) | (996, 1)
Training ExtraTreesClassifier(n_jobs=-1, random_state=42) model... [Test Accuracy=0.9716784713574097, FAR=0.032670454545454544]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.183399,1.0,1.0,0.0,0.035026,0.971678,0.970108,0.03267


Ground|Horizon=30:
Performing agnostic learning for depths (same location learning but all depths, test on a specific depth though) ...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_30Days.csv, target is T0, (#samples=4957, #features=21, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_07_30Days.csv, target is T_07, (#samples=4957, #features=21, target='T_07')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_30Days.csv, target is T_12, (#samples=4957, #features=21, target='T_12')
(11895, 22) | (11895, 21) | (11895, 1)
(992, 22) | (992, 21) | (992, 1)
Training ExtraTreesClassifier(n_jobs=-1, random_state=42) model... [Test Accuracy=0.9618786692759296, FAR=0.03857142857142857]


...

,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.200527,1.0,1.0,0.0,0.033532,0.955269,0.955455,0.045638


Ground|Horizon=30:
Performing agnostic learning for depths (same location learning but all depths, test on a specific depth though) ...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_30Days.csv, target is T0, (#samples=4957, #features=21, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_07_30Days.csv, target is T_07, (#samples=4957, #features=21, target='T_07')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_30Days.csv, target is T_12, (#samples=4957, #features=21, target='T_12')
(11895, 22) | (11895, 21) | (11895, 1)
(992, 22) | (992, 21) | (992, 1)
Training ExtraTreesClassifier(n_jobs=-1, random_state=42) model... [Test Accuracy=0.9484448734448735, FAR=0.05945945945945946]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.201661,1.0,1.0,0.0,0.030458,0.948445,0.945628,0.059459


Ground|Horizon=90:
Performing agnostic learning for depths (same location learning but all depths, test on a specific depth though) ...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_90Days.csv, target is T0, (#samples=4897, #features=21, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_07_90Days.csv, target is T_07, (#samples=4897, #features=21, target='T_07')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_90Days.csv, target is T_12, (#samples=4897, #features=21, target='T_12')
(11751, 22) | (11751, 21) | (11751, 1)
(980, 22) | (980, 21) | (980, 1)
Training ExtraTreesClassifier(n_jobs=-1, random_state=42) model... [Test Accuracy=0.9469627594627594, FAR=0.042582417582417584]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.179706,1.0,1.0,0.0,0.029155,0.946963,0.952484,0.042582


### AGNOSTIC (test from Toolik)

In [104]:
for hrz in list([0,7,30,90]):
    print(f'Ground|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=True, 
        use_second_for_testing=True,
        ftrz1=['Y', 'M', 'H', 'S', 'SNODP', 'GHTSKIN', 'TSH'], #deadhorse features
        ftrz2=['Y', 'M', 'D', 'W', 'SWGDN', 'T2M', 'GHTSKIN', 'HFLUX', 'SPEED'], #toolik features
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
# print(f'*****************************************************************************************************************')    
# for hrz in list([0,7,30,90]):
#     print(f'Mid|Horizon={hrz}:')
#     # Paths to your datasets
#     data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_07_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
#     data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T8_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

#     # Instantiate the pipeline without combining the datasets
#     pipeline = FlexibleDatasetClassificationPipeline(
#         data_path1=data_path1, 
#         data_path2=data_path2, 
#         combine_datasets=True, 
#         use_second_for_testing=True,
#         ftrz1=['Y','M','D','SNODP','EVPSOIL'], #deadhorse features
#         ftrz2=['Y', 'S', 'SNODP', 'SWLAND', 'GHTSKIN', 'SPEED', 'TLML', 'EVPSOIL', 'SLP'], #toolik features
#         seed=42,
#         test_size=0.2,
#         use_smote=False
#     )

#     # Train and test the model using a Random Forest classifier
#     pipeline.train_and_test_model(model_name="ET")

#     # Display the results
#     pipeline.display_results()
    
# print(f'*****************************************************************************************************************')    
# for hrz in list([0,7,30,90]):
#     print(f'Last|Horizon={hrz}:')
#     # Paths to your datasets
#     data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
#     data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

#     # Instantiate the pipeline without combining the datasets
#     pipeline = FlexibleDatasetClassificationPipeline(
#         data_path1=data_path1, 
#         data_path2=data_path2, 
#         combine_datasets=True, 
#         use_second_for_testing=True,
#         ftrz1=['Y','M','D','H'], #deadhorse features
#         ftrz2=['Y', 'M', 'D', 'S', 'SNODP', 'GHTSKIN', 'HFLUX', 'TLML', 'SLP'], #toolik features
#         seed=42,
#         test_size=0.2,
#         use_smote=False
#     )

#     # Train and test the model using a Random Forest classifier
#     pipeline.train_and_test_model(model_name="ET")

#     # Display the results
#     pipeline.display_results()
    
    
    

Ground|Horizon=0:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_0Days.csv, target is T0, (#samples=4987, #features=9, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_0Days.csv, target is T0, (#samples=6080, #features=9, target='T0')
Training ExtraTreesClassifier(n_jobs=-1, random_state=42) model... [Test Accuracy=0.972363290297043, FAR=0.017298735861610112]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.163103,1.0,1.0,0.0,0.031291,0.972363,0.976057,0.017299


Ground|Horizon=7:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_7Days.csv, target is T0, (#samples=4980, #features=9, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_7Days.csv, target is T0, (#samples=6071, #features=9, target='T0')
Training ExtraTreesClassifier(n_jobs=-1, random_state=42) model... [Test Accuracy=0.9515782905293184, FAR=0.030646235842771485]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.165927,1.0,1.0,0.0,0.035892,0.951578,0.95793,0.030646


Ground|Horizon=30:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_30Days.csv, target is T0, (#samples=4957, #features=9, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_30Days.csv, target is T0, (#samples=6048, #features=9, target='T0')
Training ExtraTreesClassifier(n_jobs=-1, random_state=42) model... [Test Accuracy=0.9499622424225262, FAR=0.03959731543624161]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.163366,1.0,1.0,0.0,0.031746,0.949962,0.95379,0.039597


Ground|Horizon=90:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_90Days.csv, target is T0, (#samples=4897, #features=9, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_90Days.csv, target is T0, (#samples=5988, #features=9, target='T0')
Training ExtraTreesClassifier(n_jobs=-1, random_state=42) model... [Test Accuracy=0.9544130147381913, FAR=0.030695770804911322]


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.168264,1.0,1.0,0.0,0.03118,0.954413,0.959592,0.030696


### AGNOSTIC (test from Deadhorse)

In [66]:
for hrz in list([0,7,30,90]):
    print(f'Ground|Horizon={hrz}:')
    # Paths to your datasets
    data_path2 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path1 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=True, 
        use_second_for_testing=True,
        ftrz2=['Y', 'M', 'D', 'W', 'H', 'S', 'SWGDN', 'LWGAB', 'T2M', 'GHTSKIN'], #deadhorse features
        ftrz1=['Y', 'M', 'D', 'W', 'SWGDN', 'T2M', 'GHTSKIN', 'HFLUX', 'SPEED'], #toolik features
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
print(f'*****************************************************************************************************************')    
for hrz in list([0,7,30,90]):
    print(f'Mid|Horizon={hrz}:')
    # Paths to your datasets
    data_path2 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_07_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path1 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T8_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=True, 
        use_second_for_testing=True,
        ftrz2=['Y','M','D','SNODP','EVPSOIL'], #deadhorse features
        ftrz1=['Y', 'S', 'SNODP', 'SWLAND', 'GHTSKIN', 'SPEED', 'TLML', 'EVPSOIL', 'SLP'], #toolik features
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
print(f'*****************************************************************************************************************')    
for hrz in list([0,7,30,90]):
    print(f'Last|Horizon={hrz}:')
    # Paths to your datasets
    data_path2 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path1 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=True, 
        use_second_for_testing=True,
        ftrz2=['Y','M','D','H'], #deadhorse features
        ftrz1=['Y', 'M', 'D', 'S', 'SNODP', 'GHTSKIN', 'HFLUX', 'TLML', 'SLP'], #toolik features
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
    
    

Ground|Horizon=0:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_0Days.csv, target is T0, (#samples=6080, #features=10, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_0Days.csv, target is T0, (#samples=4987, #features=10, target='T0')
Training ET model... [Test Accuracy=0.9798420037562007, FAR=0.020625415834996674]


/tmp/ipykernel_518840/69504180.py:162: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.130798,1.0,1.0,0.0,0.033214,0.979842,0.979737,0.020625


Ground|Horizon=7:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_7Days.csv, target is T0, (#samples=6071, #features=10, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_7Days.csv, target is T0, (#samples=4980, #features=10, target='T0')
Training ET model... [Test Accuracy=0.950684051008248, FAR=0.023984010659560292]


/tmp/ipykernel_518840/69504180.py:162: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


...

/tmp/ipykernel_518840/69504180.py:162: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.131636,1.0,1.0,0.0,0.034101,0.984163,0.987719,0.007097


Last|Horizon=90:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_90Days.csv, target is T16, (#samples=5988, #features=4, target='T16')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_90Days.csv, target is T_12, (#samples=4897, #features=4, target='T_12')
Training ET model... [Test Accuracy=0.9775328006313505, FAR=0.011140235910878113]


/tmp/ipykernel_518840/69504180.py:162: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.129381,1.0,1.0,0.0,0.033485,0.977533,0.982066,0.01114


### Agnostic/Train on both - unique features from both!

In [59]:
for hrz in list([0,7,30,90]):
    print(f'Ground|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=True, 
        ftrz1=['Y', 'M', 'D', 'W', 'H', 'S', 'SWGDN', 'LWGAB', 'T2M', 'GHTSKIN'], #deadhorse features
        ftrz2=['Y', 'M', 'D', 'W', 'SWGDN', 'T2M', 'GHTSKIN', 'HFLUX', 'SPEED'], #toolik features
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
print(f'*****************************************************************************************************************')    
for hrz in list([0,7,30,90]):
    print(f'Mid|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_07_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T8_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=True, 
        ftrz1=['Y','M','D','SNODP','EVPSOIL'], #deadhorse features
        ftrz2=['Y', 'S', 'SNODP', 'SWLAND', 'GHTSKIN', 'SPEED', 'TLML', 'EVPSOIL', 'SLP'], #toolik features
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
print(f'*****************************************************************************************************************')    
for hrz in list([0,7,30,90]):
    print(f'Last|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=True, 
        ftrz1=['Y','M','D','H'], #deadhorse features
        ftrz2=['Y', 'M', 'D', 'S', 'SNODP', 'GHTSKIN', 'HFLUX', 'TLML', 'SLP'], #toolik features
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
    
    

Ground|Horizon=0:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_0Days.csv, target is T0, (#samples=4987, #features=12, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_0Days.csv, target is T0, (#samples=6080, #features=12, target='T0')
Training ET model... [Test Accuracy=0.9733991931748318, FAR=0.0166333998669328]


/tmp/ipykernel_518840/3742151846.py:156: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.131337,1.0,1.0,0.0,0.033234,0.973399,0.97696,0.016633


Ground|Horizon=7:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_7Days.csv, target is T0, (#samples=4980, #features=12, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_7Days.csv, target is T0, (#samples=6071, #features=12, target='T0')
Training ET model... [Test Accuracy=0.9491325032138198, FAR=0.0313124583610926]


/tmp/ipykernel_518840/3742151846.py:156: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


...

/tmp/ipykernel_518840/3742151846.py:156: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.132651,1.0,1.0,0.0,0.032883,0.965745,0.971818,0.019355


Last|Horizon=90:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_90Days.csv, target is T_12, (#samples=4897, #features=10, target='T_12')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_90Days.csv, target is T16, (#samples=5988, #features=10, target='T16')
Training ET model... [Test Accuracy=0.9544102932679435, FAR=0.023591087811271297]


/tmp/ipykernel_518840/3742151846.py:156: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.128901,1.0,1.0,0.0,0.033383,0.95441,0.963187,0.023591


### Agnostic/Train on both - features defined by deadhorse

In [53]:
for hrz in list([0,7,30,90]):
    print(f'Ground|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=True, 
        ftrz=['Y', 'M', 'D', 'W', 'H', 'S', 'SWGDN', 'LWGAB', 'T2M', 'GHTSKIN'], #deadhorse features
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
print(f'*****************************************************************************************************************')    
for hrz in list([0,7,30,90]):
    print(f'Mid|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_07_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T8_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=True, 
        ftrz=['Y','M','D','SNODP','EVPSOIL'], #deadhorse features
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
print(f'*****************************************************************************************************************')    
for hrz in list([0,7,30,90]):
    print(f'Last|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=True, 
        ftrz=['Y','M','D','H'], #deadhorse features
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
    
    

Ground|Horizon=0:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_0Days.csv, target is T0, (#samples=4987, #features=10, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_0Days.csv, target is T0, (#samples=6080, #features=10, target='T0')
Training ET model... [Test Accuracy=0.9768774686913093, FAR=0.015302727877578177]


/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.13365,1.0,1.0,0.0,0.033284,0.976877,0.979679,0.015303


Ground|Horizon=7:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_7Days.csv, target is T0, (#samples=4980, #features=10, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_7Days.csv, target is T0, (#samples=6071, #features=10, target='T0')
Training ET model... [Test Accuracy=0.9528727327321691, FAR=0.02664890073284477]


/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


...

/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.132395,1.0,1.0,0.0,0.033793,0.983395,0.987261,0.007097


Last|Horizon=90:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_90Days.csv, target is T_12, (#samples=4897, #features=4, target='T_12')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_90Days.csv, target is T16, (#samples=5988, #features=4, target='T16')
Training ET model... [Test Accuracy=0.9752392226496991, FAR=0.015727391874180863]


/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.132502,1.0,1.0,0.0,0.033803,0.975239,0.978879,0.015727


### Agnostic/Train on both - features defined by toolik lake

In [54]:
for hrz in list([0,7,30,90]):
    print(f'Ground|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=True, 
        ftrz=['Y', 'M', 'D', 'W', 'SWGDN', 'T2M', 'GHTSKIN', 'HFLUX', 'SPEED'], #toolik features
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
print(f'*****************************************************************************************************************')    
for hrz in list([0,7,30,90]):
    print(f'Mid|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_07_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T8_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=True, 
        ftrz=['Y', 'S', 'SNODP', 'SWLAND', 'GHTSKIN', 'SPEED', 'TLML', 'EVPSOIL', 'SLP'], #toolik features
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
print(f'*****************************************************************************************************************')    
for hrz in list([0,7,30,90]):
    print(f'Last|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=True, 
        ftrz=['Y', 'M', 'D', 'S', 'SNODP', 'GHTSKIN', 'HFLUX', 'TLML', 'SLP'], #toolik features
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
    
    

Ground|Horizon=0:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_0Days.csv, target is T0, (#samples=4987, #features=9, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_0Days.csv, target is T0, (#samples=6080, #features=9, target='T0')
Training ET model... [Test Accuracy=0.972363290297043, FAR=0.017298735861610112]


/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.133715,1.0,1.0,0.0,0.033225,0.972363,0.976057,0.017299


Ground|Horizon=7:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_7Days.csv, target is T0, (#samples=4980, #features=9, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_7Days.csv, target is T0, (#samples=6071, #features=9, target='T0')
Training ET model... [Test Accuracy=0.9515782905293184, FAR=0.030646235842771485]


/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


...

/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.126732,1.0,1.0,0.0,0.033391,0.966513,0.972279,0.019355


Last|Horizon=90:
Combining datasets...
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_90Days.csv, target is T_12, (#samples=4897, #features=9, target='T_12')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_90Days.csv, target is T16, (#samples=5988, #features=9, target='T16')
Training ET model... [Test Accuracy=0.9584760213644499, FAR=0.02621231979030144]


/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.128486,1.0,1.0,0.0,0.033205,0.958476,0.964653,0.026212


### Location specific: Train on Deadhorse

In [55]:
for hrz in list([0,7,30,90]):
    print(f'T0|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=False, 
        ftrz=['Y', 'M', 'D', 'W', 'H', 'S', 'SWGDN', 'LWGAB', 'T2M', 'GHTSKIN'],
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
print(f'*****************************************************************************************************************')    
for hrz in list([0,7,30,90]):
    print(f'T0|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_07_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T8_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=False, 
        ftrz=['Y','M','D','SNODP','EVPSOIL'],
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()

    
print(f'*****************************************************************************************************************')    
for hrz in list([0,7,30,90]):
    print(f'T0|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_{hrz}Days.csv'  # Dataset 1 (Deadhorse)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_{hrz}Days.csv'  # Dataset 2 (Toolik lake)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=False, 
        ftrz=['Y','M','D','H'],
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    

T0|Horizon=0:
Loading separate datasets, Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_0Days.csv, target is T0, (#samples=4987, #features=10, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_0Days.csv, target is T0, (#samples=6080, #features=10, target='T0')
Standardizing separate datasets
Training ET model... [Test Accuracy=0.9197425381635909, FAR=0.03132832080200501]


/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.13032,1.0,1.0,0.0,0.032459,0.919743,0.934445,0.031328


T0|Horizon=7:
Loading separate datasets, Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_7Days.csv, target is T0, (#samples=4980, #features=10, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_7Days.csv, target is T0, (#samples=6071, #features=10, target='T0')
Standardizing separate datasets
Training ET model... [Test Accuracy=0.8919767909565176, FAR=0.04140526976160602]


/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


...

/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.117344,1.0,1.0,0.0,0.032502,0.766299,0.772053,0.231813


T0|Horizon=90:
Loading separate datasets, Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_90Days.csv, target is T_12, (#samples=4897, #features=4, target='T_12')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_90Days.csv, target is T16, (#samples=5988, #features=4, target='T16')
Standardizing separate datasets
Training ET model... [Test Accuracy=0.754522413668715, FAR=0.21777221526908636]


/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.120979,1.0,1.0,0.0,0.032482,0.754522,0.767661,0.217772


### Location specific: Train on Toolik

In [56]:
for hrz in list([0,7,30,90]):
    print(f'T0|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_{hrz}Days.csv'  # Dataset 1 (Toolik lake)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_{hrz}Days.csv'  # Dataset 2 (Deadhorse)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=False, 
        ftrz=['Y', 'M', 'D', 'W', 'SWGDN', 'T2M', 'GHTSKIN', 'HFLUX', 'SPEED'],
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
print(f'*****************************************************************************************************************')    
for hrz in list([0,7,30,90]):
    print(f'T0|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T8_{hrz}Days.csv'  # Dataset 1 (Toolik lake)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_07_{hrz}Days.csv'  # Dataset 2 (Deadhorse)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=False, 
        ftrz=['Y', 'S', 'SNODP', 'SWLAND', 'GHTSKIN', 'SPEED', 'TLML', 'EVPSOIL', 'SLP'],
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
print(f'*****************************************************************************************************************')    
for hrz in list([0,7,30,90]):
    print(f'T0|Horizon={hrz}:')
    # Paths to your datasets
    data_path1 = f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_{hrz}Days.csv'  # Dataset 1 (Toolik lake)
    data_path2 =  f'/home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_{hrz}Days.csv'  # Dataset 2 (Deadhorse)

    # Instantiate the pipeline without combining the datasets
    pipeline = FlexibleDatasetClassificationPipeline(
        data_path1=data_path1, 
        data_path2=data_path2, 
        combine_datasets=False, 
        ftrz=['Y', 'M', 'D', 'S', 'SNODP', 'GHTSKIN', 'HFLUX', 'TLML', 'SLP'],
        seed=42,
        test_size=0.2,
        use_smote=False
    )

    # Train and test the model using a Random Forest classifier
    pipeline.train_and_test_model(model_name="ET")

    # Display the results
    pipeline.display_results()
    
    

T0|Horizon=0:
Loading separate datasets, Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_0Days.csv, target is T0, (#samples=6080, #features=9, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_0Days.csv, target is T0, (#samples=4987, #features=9, target='T0')
Standardizing separate datasets
Training ET model... [Test Accuracy=0.9442741278280105, FAR=0.056657223796033995]


/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.130559,1.0,1.0,0.0,0.033484,0.944274,0.94451,0.056657


T0|Horizon=7:
Loading separate datasets, Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T0_7Days.csv, target is T0, (#samples=6071, #features=9, target='T0')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T0_7Days.csv, target is T0, (#samples=4980, #features=9, target='T0')
Standardizing separate datasets
Training ET model... [Test Accuracy=0.939436099003736, FAR=0.03551136363636364]


/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


...

/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.130012,1.0,1.0,0.0,0.032287,0.865948,0.882605,0.105405


T0|Horizon=90:
Loading separate datasets, Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Toolik_T16_90Days.csv, target is T16, (#samples=5988, #features=9, target='T16')
Loading data from /home/mohamed.ahajjam/FreezeThaw/FTstatesDataset_Deadhorse_T_12_90Days.csv, target is T_12, (#samples=4897, #features=9, target='T_12')
Standardizing separate datasets
Training ET model... [Test Accuracy=0.8621031746031746, FAR=0.125]


/tmp/ipykernel_518840/1091888499.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([self.results, pd.DataFrame({


,Model,Train Time (s),Train wAccuracy,Train f1w,Train FAR,Test Time (s),Test wAccuracy,Test f1w,Test FAR
0,ET,0.129656,1.0,1.0,0.0,0.032821,0.862103,0.872144,0.125
